# Retail Store Sales Analysis
This notebook provides an analysis of retail store and a system to clasify differents types of clients based on their purchasing behavior.

## Importing Libraries

In [4]:
import pandas as pd
import numpy as np

## Load the dataset

In [5]:
df = pd.read_csv('retail_store_sales.csv')

## Explore the data

In [7]:
df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


As we can see, the dataset contains information about the sales of a retail store, including the date of purchase, the type of client, the amount spent, and the product category. Exploring the cuantity of non-null values in each column, we can see that the dataset contains missing values in the following columns: `Item`, `Price Per Unit`, `Quantity`, `Total Spent` and `Discount Applied` .

### Transaction ID

In [9]:
# Lets have a look at the first 50 rows of the dataset to see if the Transaction ID have a form of codification
df['Transaction ID'].head(50)

0     TXN_6867343
1     TXN_3731986
2     TXN_9303719
3     TXN_9458126
4     TXN_4575373
5     TXN_7482416
6     TXN_3652209
7     TXN_1372952
8     TXN_9728486
9     TXN_2722661
10    TXN_8776416
11    TXN_5422631
12    TXN_5874772
13    TXN_4413070
14    TXN_2490363
15    TXN_1809665
16    TXN_7563311
17    TXN_9634894
18    TXN_4396807
19    TXN_4206593
20    TXN_9939063
21    TXN_8685338
22    TXN_6547964
23    TXN_3314099
24    TXN_9980736
25    TXN_3481599
26    TXN_9057609
27    TXN_1599706
28    TXN_1598860
29    TXN_7138501
30    TXN_5444504
31    TXN_4353295
32    TXN_1543244
33    TXN_1494700
34    TXN_1621497
35    TXN_6398436
36    TXN_9065245
37    TXN_6963438
38    TXN_8312797
39    TXN_5836005
40    TXN_4223250
41    TXN_1019817
42    TXN_1110407
43    TXN_6276518
44    TXN_4740738
45    TXN_9587232
46    TXN_6404316
47    TXN_9331642
48    TXN_9729422
49    TXN_8425168
Name: Transaction ID, dtype: object

As we can see, It looks like the Transaction ID is a combination of the leters TXN, a _ and a number. We can make a regex to see if this code is followed by the rest of the dataset.

In [14]:
#To show which row do not follow the expected format of Transaction ID
df[~df['Transaction ID'].astype(str).str.match(r"^TXN_\d{7}$")]['Transaction ID'].nunique()

0

It seems that the Transaction ID is consistent!!

### Customer ID
lets do the same fo customer ID

In [15]:
df[~df['Customer ID'].astype(str).str.match(r"^CUST_\d{2}$")]['Customer ID'].nunique()

0

Perfect, now we can see that the Transaction ID and Customer ID columns have a specific format. Now we can see who are the top 10 customers by the number of transactions they made.

In [ ]:
#Top 10 customers by number of transactions
df['Customer ID'].value_counts().sort_values(ascending=False).head(10)

Customer ID
CUST_05    544
CUST_24    543
CUST_13    534
CUST_08    533
CUST_09    519
CUST_15    519
CUST_16    515
CUST_23    513
CUST_20    507
CUST_18    507
Name: count, dtype: int64

### Category

In [31]:
categories_of_products = df['Category'].value_counts().sort_values(ascending=False).index


In [33]:
for category in categories_of_products:
    print(f"Category: {category}, Number of unique items: {df[df['Category'] == category]['Item'].value_counts().shape}")

Category: Electric household essentials, Number of unique items: (25,)
Category: Furniture, Number of unique items: (25,)
Category: Food, Number of unique items: (25,)
Category: Milk Products, Number of unique items: (25,)
Category: Butchers, Number of unique items: (25,)
Category: Beverages, Number of unique items: (25,)
Category: Computers and electric accessories, Number of unique items: (25,)
Category: Patisserie, Number of unique items: (25,)


The number of items of each category is always the same 

In [38]:
for category in categories_of_products:
    temp = df[df['Category'] == category]
    price_variation = (
        temp.groupby('Item')['Price Per Unit']
        .nunique()
        .reset_index(name='Unique Prices')
    )
    items_with_price_change = price_variation[price_variation['Unique Prices'] > 1]

    print(f"Category: {category}")
    if not items_with_price_change.empty:
        print("Items with price variation:")
        print(items_with_price_change)
    else:
        print("No price variation detected.\n")


Category: Electric household essentials
No price variation detected.

Category: Furniture
No price variation detected.

Category: Food
No price variation detected.

Category: Milk Products
No price variation detected.

Category: Butchers
No price variation detected.

Category: Beverages
No price variation detected.

Category: Computers and electric accessories
No price variation detected.

Category: Patisserie
No price variation detected.



As we can see, the price of the product is always the same, so if we have a missing value in either the category or Total Spent, we can fill it if we have the other two columns.

In [ ]:
# Check for NaN values in 'Price Per Unit'
print(f'price per unit is NaN: {df[df["Price Per Unit"].isna()].shape}')
print(f'price per unit is NaN and Total Spent too : {df[(df["Price Per Unit"].isna()) & (df["Total Spent"].isna())].shape}')

price per unit is NaN: (609, 11)
price per unit is NaN and : (0, 11)


In [41]:
df[df["Price Per Unit"].isna()]

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
5,TXN_7482416,CUST_09,Patisserie,NaN,NaN,10.0,200.0,Credit Card,Online,2023-11-30,NaN
11,TXN_5422631,CUST_09,Milk Products,NaN,NaN,8.0,52.0,Digital Wallet,In-store,2025-01-12,True
17,TXN_9634894,CUST_15,Milk Products,NaN,NaN,10.0,275.0,Digital Wallet,Online,2022-04-17,NaN
21,TXN_8685338,CUST_15,Milk Products,NaN,NaN,3.0,105.0,Credit Card,In-store,2023-10-29,NaN
32,TXN_1543244,CUST_20,Food,NaN,NaN,8.0,196.0,Credit Card,Online,2024-10-25,True
...,...,...,...,...,...,...,...,...,...,...,...
12435,TXN_5147764,CUST_01,Furniture,NaN,NaN,5.0,32.5,Credit Card,Online,2023-09-06,True
12457,TXN_1352194,CUST_17,Electric household essentials,NaN,NaN,4.0,86.0,Credit Card,Online,2023-02-26,NaN
12477,TXN_5625684,CUST_22,Computers and electric accessories,NaN,NaN,4.0,80.0,Cash,In-store,2022-11-09,True
12491,TXN_7894525,CUST_23,Butchers,NaN,NaN,1.0,26.0,Credit Card,Online,2023-01-31,True


Now lets explore the Total spent column. If we dont have a value here we can try to fill it with the price per unit and the quantity. And if we dont have the value of the Quantity, we can fill it like the customer only bought one item.

In [42]:
df[df["Total Spent"].isna()]

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
7,TXN_1372952,CUST_21,Furniture,NaN,33.5,NaN,NaN,Digital Wallet,In-store,2024-04-02,True
15,TXN_1809665,CUST_14,Beverages,NaN,24.5,NaN,NaN,Credit Card,In-store,2022-05-11,NaN
19,TXN_4206593,CUST_01,Furniture,NaN,35.0,NaN,NaN,Digital Wallet,Online,2025-01-13,False
25,TXN_3481599,CUST_05,Furniture,NaN,39.5,NaN,NaN,Cash,Online,2022-09-08,False
34,TXN_1621497,CUST_06,Patisserie,NaN,23.0,NaN,NaN,Cash,In-store,2023-02-18,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12527,TXN_1069238,CUST_23,Food,NaN,5.0,NaN,NaN,Digital Wallet,In-store,2022-08-13,False
12552,TXN_4823896,CUST_05,Milk Products,NaN,8.0,NaN,NaN,Cash,In-store,2022-07-21,False
12556,TXN_4397672,CUST_04,Beverages,NaN,41.0,NaN,NaN,Credit Card,Online,2024-11-28,True
12562,TXN_7422454,CUST_07,Butchers,NaN,33.5,NaN,NaN,Cash,Online,2023-04-15,NaN


In [43]:
df['Payment Method'].value_counts()

Payment Method
Cash              4310
Digital Wallet    4144
Credit Card       4121
Name: count, dtype: int64

In [44]:
df['Location'].value_counts()

Location
Online      6354
In-store    6221
Name: count, dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB
